In [1]:
import geopandas as gpd
import pandas as pd
import os
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error

In [2]:
cells = gpd.read_file('global-inputs/HR6-change-cells-aoi.gpkg')

cell_ids = cells.hex_id.tolist()

In [3]:
shoreline_files = pd.Series(sorted(glob("data/HR6/*/shoreline_timeseries_tidal_correction.csv")))

In [4]:
def calc_trends(file):
    df  = pd.read_csv(file)
    try:
        df.date = pd.to_datetime(df.date)
    except:
        print(file)
    df.index = (df.date - df.date.min()).dt.days / 365.25
    df.drop(columns=['date', 'Unnamed: 0', 'corrected_IW_shoreline_position', 'EOV_shoreline_chg'], inplace=True, errors='ignore')
    trends = {}
    for col in df.columns:
        sub_df = df[col].dropna()
        
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1,1)
        y = sub_df
        linear_trend = LinearRegression().fit(x,y)
        pred = linear_trend.predict(x)

        trends['index'] = file.split('/')[2]
        if col == 'normalised_IW_shoreline_position':
            trends["IW shoreline trend"] = linear_trend.coef_[0]
            trends["IW shoreline intercept"] = linear_trend.intercept_
            trends["IW shoreline n_points"] = len(df[col])
            trends["IW shoreline n_points_nonan"] = len(sub_df)
            trends["IW shoreline r2_score"] = r2_score(y, pred)
            trends["IW shoreline mae"] = mean_absolute_error(y, pred)
            trends["IW shoreline mse"] = mean_squared_error(y, pred)
            trends["IW shoreline rmse"] = root_mean_squared_error(y, pred)
        else:
            trends["EOV shoreline trend"] = linear_trend.coef_[0]
            trends["EOV shoreline intercept"] = linear_trend.intercept_
            trends["EOV shoreline n_points"] = len(df[col])
            trends["EOV shoreline n_points_nonan"] = len(sub_df)
            trends["EOV shoreline r2_score"] = r2_score(y, pred)
            trends["EOV shoreline mae"] = mean_absolute_error(y, pred)
            trends["EOV shoreline mse"] = mean_squared_error(y, pred)
            trends["EOV shoreline rmse"] = root_mean_squared_error(y, pred)

    return pd.DataFrame([trends])
    

In [5]:
trends = pd.concat([calc_trends(f) for f in shoreline_files])

trends.dropna(inplace=True)

trends

/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


,index,IW shoreline trend,IW shoreline intercept,IW shoreline n_points,IW shoreline n_points_nonan,IW shoreline r2_score,IW shoreline mae,IW shoreline mse,IW shoreline rmse,EOV shoreline trend,EOV shoreline intercept,EOV shoreline n_points,EOV shoreline n_points_nonan,EOV shoreline r2_score,EOV shoreline mae,EOV shoreline mse,EOV shoreline rmse
0,86ba24017ffffff,0.628636,4.655082,179.0,179.0,0.038160,12.202930,207.494892,14.404683,-0.455192,-0.508700,179.0,179.0,0.005899,22.395659,727.414725,26.970627
0,86ba24037ffffff,1.045480,-6.332582,167.0,167.0,0.351077,6.947731,67.527685,8.217523,1.589718,-20.802563,167.0,167.0,0.429079,5.574640,112.393204,10.601566
0,86ba24067ffffff,0.407965,15.650082,227.0,227.0,0.009428,9.913116,385.802461,19.641855,0.743270,-19.038936,227.0,227.0,0.012396,24.093837,971.097115,31.162431
0,86ba2414fffffff,0.337332,5.439120,195.0,195.0,0.017055,6.518992,100.313399,10.015658,0.616934,-15.471110,195.0,195.0,0.079823,7.133070,67.108520,8.191979
0,86ba2415fffffff,0.420564,0.448312,312.0,312.0,0.006715,14.670830,710.373364,26.652830,-0.149873,-12.764045,312.0,312.0,0.000445,26.209901,1370.731517,37.023391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,86dab3707ffffff,0.770987,23.240949,238.0,238.0,0.066371,18.928175,487.922176,22.088961,0.179675,13.654287,238.0,238.0,0.073785,3.432934,23.647125,4.862831
0,86dab370fffffff,0.269206,27.797196,213.0,213.0,0.013518,15.707413,317.843751,17.828173,-0.172802,19.689800,213.0,213.0,0.004657,13.725087,383.570035,19.584944
0,86dab3737ffffff,1.917189,-13.477238,173.0,173.0,0.634656,7.831589,121.290323,11.013189,0.061760,-1.092161,173.0,173.0,0.038704,0.996285,5.430655,2.330377
0,86dab3757ffffff,-0.745551,23.194218,208.0,208.0,0.090887,14.960857,321.908315,17.941804,0.331800,9.903389,208.0,208.0,0.071214,7.624934,83.131319,9.117638


In [6]:
trends_file_path = 'shoreline_trends_by_cell.geojson'
# if os.path.exists(trends_file_path):
#     cell_trends = gpd.read_file('shoreline_trends_by_cell.geojson')
#     cell_trends.to_crs("EPSG:4326", inplace=True)
#     cell_trends.update(trends)
#     cell_trends.to_file(trends_file_path)
# else:
cell_trends = cells.merge(trends, on='index')
cell_trends.to_crs("EPSG:4326", inplace=True)
cell_trends.to_file(trends_file_path)
    
# cell_trends['hex_id'].to_csv("global-inputs/valid_cells.csv")

cell_trends

,index,hex_id,parent_id,sediment_area_ha,geometry,IW shoreline trend,IW shoreline intercept,IW shoreline n_points,IW shoreline n_points_nonan,IW shoreline r2_score,...,IW shoreline mse,IW shoreline rmse,EOV shoreline trend,EOV shoreline intercept,EOV shoreline n_points,EOV shoreline n_points_nonan,EOV shoreline r2_score,EOV shoreline mae,EOV shoreline mse,EOV shoreline rmse
0,86bb5e18fffffff,86bb5e18fffffff,85bb5e1bfffffff,16.76,"POLYGON ((173.94469 -35.12657, 173.94025 -35.0...",0.226193,-3.483359,141.0,141.0,0.017436,...,104.072183,10.201577,1.242516,-22.379807,141.0,141.0,0.060747,22.273659,861.622758,29.353411
1,86bb5e19fffffff,86bb5e19fffffff,85bb5e1bfffffff,20.00,"POLYGON ((174.01618 -35.12027, 174.01171 -35.0...",0.480886,2.732576,86.0,86.0,0.082123,...,55.494168,7.449441,0.523784,-5.437775,86.0,86.0,0.048198,10.084683,116.323886,10.785355
2,86bb58027ffffff,86bb58027ffffff,85bb5803fffffff,97.20,"POLYGON ((172.71095 -34.47891, 172.70704 -34.4...",0.826371,-0.681890,7.0,7.0,0.081622,...,29.049672,5.389775,3.016152,8.967772,7.0,7.0,0.070048,18.404964,456.612787,21.368500
3,86bb72b4fffffff,86bb72b4fffffff,85bb72b7fffffff,14.08,"POLYGON ((173.87492 -39.18614, 173.87027 -39.1...",-0.114118,22.923172,156.0,156.0,0.000483,...,654.064682,25.574688,1.813691,-58.763834,156.0,156.0,0.049544,34.152435,1532.068823,39.141651
4,86bb2bae7ffffff,86bb2bae7ffffff,85bb2baffffffff,44.20,"POLYGON ((175.14401 -40.80153, 175.13871 -40.7...",1.450422,2.098858,100.0,100.0,0.188674,...,471.941215,21.724208,0.888505,-8.358124,100.0,100.0,0.143339,11.039413,246.138043,15.688787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,86da9285fffffff,86da9285fffffff,85da9287fffffff,2.00,"POLYGON ((171.98283 -41.59482, 171.97888 -41.5...",-0.324312,6.715930,261.0,261.0,0.015637,...,417.435052,20.431227,0.081759,10.785876,261.0,261.0,0.073628,1.301265,5.302357,2.302685
631,86da94ab7ffffff,86da94ab7ffffff,85da94abfffffff,50.36,"POLYGON ((172.82222 -43.2102, 172.81779 -43.17...",-0.194254,-10.164818,175.0,175.0,0.007225,...,345.565446,18.589391,0.330989,-2.666553,175.0,175.0,0.203491,4.341013,28.577840,5.345825
632,86da91d8fffffff,86da91d8fffffff,85da91dbfffffff,5.36,"POLYGON ((171.05489 -42.73298, 171.05131 -42.6...",-0.775604,11.781211,242.0,242.0,0.056909,...,436.989204,20.904287,0.092400,-0.340279,242.0,242.0,0.292600,0.798689,0.904800,0.951210
633,86bb299afffffff,86bb299afffffff,85bb299bfffffff,11.68,"POLYGON ((174.2819 -41.84778, 174.27691 -41.81...",2.808384,-40.001057,332.0,332.0,0.290624,...,662.846894,25.745813,-4.459806,20.987040,332.0,332.0,0.181982,42.935272,3078.382919,55.483177
